In [1]:
!wget http://www.iapr-tc11.org/dataset/4NSigComp2010/Dataset_4NSigComp2010.zip


--2021-04-29 15:04:03--  http://www.iapr-tc11.org/dataset/4NSigComp2010/Dataset_4NSigComp2010.zip
Resolving www.iapr-tc11.org (www.iapr-tc11.org)... 157.16.221.56
Connecting to www.iapr-tc11.org (www.iapr-tc11.org)|157.16.221.56|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1522758855 (1.4G) [application/zip]
Saving to: ‘Dataset_4NSigComp2010.zip’

Dataset_4NSigComp20 100%[===================>]   1.42G  18.8MB/s    in 2m 11s  

2021-04-29 15:06:15 (11.1 MB/s) - ‘Dataset_4NSigComp2010.zip’ saved [1522758855/1522758855]



In [3]:
!7z e -p"I hereby accept the 4NSigComp2010 disclaimer." Dataset_4NSigComp2010.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1522758855 bytes (1453 MiB)

Extracting archive: Dataset_4NSigComp2010.zip
--
Path = Dataset_4NSigComp2010.zip
Type = zip
Physical Size = 1522758855

  0%    
Would you like to replace the existing file:
  Path:     ./Correct Answer Key.pdf
  Size:     8499 bytes (9 KiB)
  Modified: 2010-05-12 12:24:04
with the file from archive:
  Path:     Dataset_4NSigComp2010/TestSet/Correct Answer Key.pdf
  Size:     8499 bytes (9 KiB)
  Modified: 2010-05-12 12:24:04
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% 2 - Dataset_4NSigComp2010/TestSet/Correct Answer Key.pdf                                                             

In [41]:
from PIL import Image, ImageDraw
import numpy as np

c = []
t = []
r = []
b = []
s = []


In [42]:
def BWImage(image_array):
    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            if image_array[i, j] < 110:
                image_array[i, j] = 0
            else:
                image_array[i, j] = 255

    BWImage = Image.fromarray(image_array)
    return BWImage

In [43]:
def boundingBox(BWImage):
    width, height = BWImage.size
    left = width
    right = 0
    top = height
    bottom = 0

    for x in range(width):
        for y in range(height):
            color = BWImage.getpixel((x, y))
            if color == 0:
                if x > right:
                    right = x
                if x < left:
                    left = x
                if y > bottom:
                    bottom = y
                if y < top:
                    top = y

    img_shape = [(left, top), (right, bottom)]
    bounding_box = ImageDraw.Draw(BWImage)
    bounding_box.rectangle(img_shape, outline="#000000")
    coordinates = (left, right, top, bottom)
    return coordinates

In [44]:
def centroid(img, l, r, t, b):
    cx = 0
    cy = 0
    n = 0
    for x in range(int(l), int(r)):
        for y in range(int(t), int(b)):
            color = img.getpixel((x, y))
            if color == 0:
                cx = cx + x
                cy = cy + y
                n = n + 1
    if n == 0:
        cx = cx / 1
        cy = cy / 1
    else:
        cx = cx / n
        cy = cy / n
    return (cx, cy)

In [45]:
def findTransitions(image, l, r, t, b):
    prev = image.getpixel((l, t))
    n = 0
    for x in range(int(l) + 1, int(r)):
        for y in range(int(t) + 1, int(b)):
            curr = image.getpixel((x, y))
            if curr == 255 and prev == 0:
                n = n + 1
            prev = curr
    return n

In [46]:
def findRatio(l, r, t, b):
    ratio = (r - l)/(b - t)
    return ratio

In [47]:
def findBlacks(image, l, r, t, b):
    count = 0
    for x in range(int(l), int(r)):
        for y in range(int(t), int(b)):
            if image.getpixel((x, y)) == 0:
                count += 1
    return count

In [48]:
def findNormSize(l, r, t, b, bPixels):
    if bPixels == 0:
        bPixels = 1
    s = ((r - l) * (b - t))/bPixels
    return s


In [49]:
def split(image, left, right, top, bottom, depth = 0):
    cx, cy = centroid(image, left, right, top, bottom)
    if depth < 3:
        split(image, left, cx, top, cy, depth + 1)
        split(image, cx, right, top, cy, depth + 1)
        split(image, left, cx, cy, bottom, depth + 1)
        split(image, cx, right, cy, bottom, depth + 1)
    else:
        c.append((cx, cy))
        t.append(findTransitions(image, left, right, top, bottom))
        r.append(findRatio(left, right, top, bottom))
        b.append(findBlacks(image, left, right, top, bottom))
        s.append(findNormSize(left, right, top, bottom, b[-1]))

        c_array = np.array(c)
        np.savetxt('centroids.txt', c_array, delimiter =', ')
        t_array = np.array(t)
        np.savetxt('transitions.txt', t_array, delimiter=', ')
        r_array = np.array(r)
        np.savetxt('ratios.txt', r_array, delimiter=', ')
        b_array = np.array(b)
        np.savetxt('blackPixels.txt', b_array, delimiter=', ')
        s_array = np.array(s)
        np.savetxt('normalizedSizes.txt', s_array, delimiter=', ')

In [50]:


if __name__ == '__main__':
    image = Image.open("Q028.png").convert('L')
    image_array = np.array(image)
    BW_image = BWImage(image_array)
    coordinates = boundingBox(BW_image)
    split(BW_image, *coordinates, 0)


In [51]:
centroid = open("centroids.txt", "r")
centroidContents = centroid.read()
print(centroidContents) 


1.701292735042734989e+02, 1.697649572649572747e+01
3.735846817691477781e+02, 1.743581445523193096e+01
4.569395017793594604e+01, 8.503914590747331204e+01
3.791593038821954451e+02, 8.965461847389558159e+01
9.715006134969324876e+02, 1.582453987730061407e+01
1.243473917869034494e+03, 1.384128745837957908e+01
1.002386046511627910e+03, 7.274263565891472183e+01
1.132331439393939490e+03, 7.503598484848484418e+01
1.821124497991967814e+02, 1.440040160642570299e+02
5.542165775401069823e+02, 1.420178253119429712e+02
9.867198838896952395e+01, 2.228679245283018986e+02
5.636542288557213851e+02, 2.145713101160862379e+02
1.032630824372759889e+03, 1.482267025089605852e+02
1.578988165680473458e+03, 1.481627218934911241e+02
1.026721017907634405e+03, 2.262997172478793573e+02
1.583010752688172033e+03, 1.963010752688172147e+02
1.724898951382268933e+03, 4.356339370829361002e+01
2.099500000000000000e+03, 9.000000000000000000e+00
1.709952674897119323e+03, 9.479218106995884341e+01
2.193482261640798242e+03, 1.110

In [52]:
transitions = open("transitions.txt", "r")
transitionsContents = transitions.read()
print(transitionsContents) 

1.340000000000000000e+02
8.700000000000000000e+01
7.800000000000000000e+01
8.600000000000000000e+01
5.900000000000000000e+01
5.000000000000000000e+01
7.500000000000000000e+01
3.100000000000000000e+01
1.440000000000000000e+02
4.400000000000000000e+01
5.200000000000000000e+01
4.700000000000000000e+01
1.010000000000000000e+02
3.700000000000000000e+01
6.900000000000000000e+01
3.200000000000000000e+01
1.190000000000000000e+02
1.000000000000000000e+00
8.900000000000000000e+01
7.700000000000000000e+01
1.000000000000000000e+00
2.300000000000000000e+01
1.600000000000000000e+01
3.800000000000000000e+01
1.300000000000000000e+01
4.800000000000000000e+01
1.200000000000000000e+02
7.600000000000000000e+01
1.000000000000000000e+01
5.000000000000000000e+01
1.310000000000000000e+02
8.000000000000000000e+01
1.180000000000000000e+02
1.760000000000000000e+02
1.140000000000000000e+02
1.790000000000000000e+02
1.150000000000000000e+02
3.900000000000000000e+01
0.000000000000000000e+00
6.800000000000000000e+01


In [53]:
ratios = open("ratios.txt", "r")
ratiosContents = ratios.read()
print(ratiosContents) 

6.645173291395040316e+00
1.115151626597226553e+01
3.618870643420939004e+00
6.072963499208625215e+00
1.412655150600416221e+01
1.708542670342611558e+01
5.525925194030638643e+00
6.683357210788368974e+00
5.949769514327524433e+00
4.508904853907625743e+00
5.358649856416363377e+00
4.060937535446365487e+00
6.879856664822280976e+00
6.880491789955145876e+00
6.983861879473015399e+00
6.984506606016053887e+00
4.619444611103450171e+00
6.519571034189068470e+00
3.789161349980825655e+00
5.347765513158246442e+00
5.773329004514957674e+00
3.463317493679367587e+00
5.177838734405634469e+00
3.106093460860066990e+00
7.269617059867454323e+00
3.665008120026429062e+00
1.371436564037036909e+01
6.914155315064876461e+00
5.548755132671393753e+00
3.390323734991324667e+00
1.267742930195417372e+01
7.745987781658900673e+00
6.908983812351626952e+00
8.146190589618038658e+00
4.088968202434400290e+00
4.821188643164711074e+00
6.601916997137369236e+00
1.032470256083676752e+01
3.988991073842360624e+00
6.238361732374603896e+00


In [54]:
blackPixels = open("blackPixels.txt", "r")
blackPixelsContents = blackPixels.read()
print(blackPixelsContents) 

9.360000000000000000e+02
9.270000000000000000e+02
5.620000000000000000e+02
7.470000000000000000e+02
8.150000000000000000e+02
9.010000000000000000e+02
6.450000000000000000e+02
5.280000000000000000e+02
9.960000000000000000e+02
1.122000000000000000e+03
6.890000000000000000e+02
1.206000000000000000e+03
1.116000000000000000e+03
3.380000000000000000e+02
1.061000000000000000e+03
1.860000000000000000e+02
1.049000000000000000e+03
4.140000000000000000e+02
9.720000000000000000e+02
9.020000000000000000e+02
3.850000000000000000e+02
4.910000000000000000e+02
6.110000000000000000e+02
4.860000000000000000e+02
5.500000000000000000e+01
9.530000000000000000e+02
8.900000000000000000e+02
8.270000000000000000e+02
4.710000000000000000e+02
5.540000000000000000e+02
9.830000000000000000e+02
1.313000000000000000e+03
1.153000000000000000e+03
2.148000000000000000e+03
1.597000000000000000e+03
1.648000000000000000e+03
1.065000000000000000e+03
8.260000000000000000e+02
8.000000000000000000e+00
9.500000000000000000e+02


In [55]:
normalizedSizes = open("normalizedSizes.txt", "r")
normalizedSizesContents = normalizedSizes.read()
print(normalizedSizesContents) 

9.869745503521880181e+00
1.672359648870170545e+01
3.018413529302637599e+01
3.810851835598372617e+01
1.532570102950036706e+01
1.676651644796619323e+01
4.950503810096979862e+01
7.314166856390907867e+01
2.404592357844795103e+01
1.617628809316479632e+01
3.859458191820151285e+01
1.670972042813187741e+01
2.804404936446239915e+01
9.260369330997494330e+01
2.905850601357580842e+01
1.657737693608342795e+02
1.771974276410062998e+01
6.336680605885154449e+01
2.331381261941418970e+01
3.545702441429251905e+01
6.642731489210628126e+01
3.124582056996700175e+01
4.667066664417972532e+01
3.519775343071948726e+01
5.519221301736851046e+02
1.605872288921435498e+01
1.807949523406152537e+01
9.809213907031960389e+00
5.571843123120516594e+01
2.894380562233468979e+01
1.168505153161862076e+01
5.345213222262261255e+00
1.463857118960088144e+01
9.264758663562606245e+00
1.785761773448541234e+01
2.040382652287489407e+01
1.554140148713210934e+01
3.133770800625513075e+01
3.424181653513895981e+03
4.509523630422369678e+01
